<a href="https://colab.research.google.com/github/mertege/FMCW-Data-Classification-/blob/main/FMCW_Data_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spectrogram'lar üzerinden CNN koşturuluyor.
# 5-fold koşturuldu ve training datasına augmentation yapıldı.
# mean test accuracy = 0.84
# mean f score = 0.80
# training_time = 474.91

In [35]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support

In [3]:
pip install mat73

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import mat73
spectrograms_shuffled = mat73.loadmat('/content/drive/MyDrive/spectrograms_shuffled.mat')
spectrograms_shuffled = spectrograms_shuffled['spectrograms_shuffled']
spectrograms_shuffled = np.transpose(spectrograms_shuffled, (2, 0, 1))
spectrograms_label_shuffled = scipy.io.loadmat('/content/drive/MyDrive/spectrograms_label_shuffled.mat')
spectrograms_label_shuffled = spectrograms_label_shuffled['spectrograms_label_shuffled']  

In [5]:
spectrograms_shuffled_downscaled = np.zeros((170,50,412))
for ii in range(spectrograms_shuffled.shape[0]):
  aa = np.float32(spectrograms_shuffled[ii,:,:])
  img = cv2.cvtColor(aa, cv2.COLOR_GRAY2BGR)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  spectrograms_shuffled_downscaled[ii,:,:] = cv2.resize(img_gray, dsize=(412, 50), interpolation=cv2.INTER_CUBIC)


In [6]:
del spectrograms_shuffled

In [7]:
print(spectrograms_shuffled_downscaled.shape[1:])
spectrograms_shuffled_downscaled = spectrograms_shuffled_downscaled[:,:,:,np.newaxis] 
print(spectrograms_shuffled_downscaled.shape[1:])

(50, 412)
(50, 412, 1)


In [41]:
t = time.time()
# ---------------- Augmente and shuffle (train and test) data data ----------------
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
])
# ---------------- Parameters ----------------
test_accuracy_per_fold = []
f1_score_per_fold = []
repeat_of_augmentation = 9
size_of_validation = 40
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
epoch_number = 60
batch_size = 32

for train, test in kfold.split(spectrograms_shuffled_downscaled,spectrograms_label_shuffled):   
  randomlist_for_validation_indx = test
  randomlist_for_train_indx = train
  # test data
  spectrograms_shuffled_downscaled_test = spectrograms_shuffled_downscaled[randomlist_for_validation_indx,:,:,:]
  spectrograms_label_shuffled_test = spectrograms_label_shuffled[randomlist_for_validation_indx,:]
  #train data
  spectrograms_shuffled_downscaled_train = spectrograms_shuffled_downscaled[randomlist_for_train_indx,:,:,:]
  spectrograms_label_shuffled_train = spectrograms_label_shuffled[randomlist_for_train_indx,:]

  size_of_samples = spectrograms_shuffled_downscaled_train.shape[0]
  augmented_image = np.zeros((size_of_samples*repeat_of_augmentation,spectrograms_shuffled_downscaled_train.shape[1],spectrograms_shuffled_downscaled_train.shape[2],1))
  spectrograms_label_shuffled_concat = spectrograms_label_shuffled_train
  for kk in range(repeat_of_augmentation):
    spectrograms_label_shuffled_concat = np.concatenate((spectrograms_label_shuffled_concat, spectrograms_label_shuffled_train),axis=0)
    for ii in range(size_of_samples):
      augmented_image[size_of_samples*kk+ii,:,:,:] = data_augmentation(spectrograms_shuffled_downscaled_train[ii,:,:,:])

  augmented_image = np.concatenate((augmented_image,spectrograms_shuffled_downscaled_train),axis=0)
  layer = Normalization(axis=None)
  layer.adapt(spectrograms_shuffled_downscaled_test)
  spectrograms_shuffled_downscaled_test = layer(spectrograms_shuffled_downscaled_test)
  layer = Normalization(axis=None)
  layer.adapt(augmented_image)
  augmented_image = layer(augmented_image)

  # ---------------- Neural Network Architecture ----------------
  model = Sequential()

  model.add(Conv2D(8, (3, 3), input_shape=augmented_image.shape[1:]))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(16, (3, 3), input_shape=augmented_image.shape[1:]))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(32, (3, 3)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))


  # model.add(Conv2D(64, (3, 3)))
  # model.add(BatchNormalization())
  # model.add(Activation('relu'))
  # model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())  
  model.add(Dense(32))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))

  print(model.summary())
  # ---------------- Compile and Fit ----------------
  model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

  history = model.fit((augmented_image),(spectrograms_label_shuffled_concat),
                  epochs=epoch_number,
                  batch_size=batch_size,
                  shuffle = True,
                  validation_data = ((spectrograms_shuffled_downscaled_test) , (spectrograms_label_shuffled_test)))
  
  test_loss, test_accuracy  = model.evaluate((spectrograms_shuffled_downscaled_test),(spectrograms_label_shuffled_test),
                batch_size=batch_size)
  # ---------------- Get Test Results ----------------
  y_test_predicted = model.predict((spectrograms_shuffled_downscaled_test), batch_size=batch_size)
  # ----- Binarize y_test_predicted values -----
  y_test_predicted_binary = np.zeros(y_test_predicted.size)
  for ii in range(y_test_predicted.size):
    if y_test_predicted[ii] < 0.5:
      y_test_predicted_binary[ii] = 0
    else:
      y_test_predicted_binary[ii] = 1
  
  test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(spectrograms_label_shuffled_test, y_test_predicted_binary, average='macro')

  test_accuracy_per_fold.append(test_accuracy)
  f1_score_per_fold.append(test_f1_score)

print(sum(test_accuracy_per_fold)/num_folds)
print(sum(f1_score_per_fold)/num_folds)
elapsed = time.time() - t
print(elapsed)

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 48, 410, 8)        80        
_________________________________________________________________
batch_normalization_42 (Batc (None, 48, 410, 8)        32        
_________________________________________________________________
activation_70 (Activation)   (None, 48, 410, 8)        0         
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 24, 205, 8)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 22, 203, 16)       1168      
_________________________________________________________________
batch_normalization_43 (Batc (None, 22, 203, 16)       64        
_________________________________________________________________
activation_71 (Activation)   (None, 22, 203, 16)     

In [ ]:
# Test Amaçlı
model.predict(augmented_image)

In [ ]:
# Test Amaçlı
model.predict(spectrograms_shuffled_downscaled_test)